### preprocess

In [1]:
import pandas as pd
import numpy as np
import random
import itertools
from tqdm.notebook import tqdm
import re
# from mecab import MeCab

In [2]:
train_df = pd.read_csv('../train.csv')
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [3]:
def cleansing(train_df):

    # 공백이 3개 이상인 경우 한개로 바꿈
    for i in tqdm(range(len(train_df)), total=len(train_df)):
        for answer in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            train_df.loc[i, answer] = re.sub(r'\s{2,}', ' ', train_df.loc[i, answer])

    # Function to remove URLs and specific patterns (resourceName: and special characters) from a string
    def remove_url_and_pattern(text):
        # Remove URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        # Remove specific pattern and special characters
        text = re.sub(r'\(resourceName:\s?', '', text)
        # Remove special characters (keeping Korean characters, numbers, and basic punctuation)
        return text
    answer_columns = ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']
    for column in answer_columns:
        train_df[column] = train_df[column].apply(remove_url_and_pattern)
    # 장점, 단점이 질문에 있지만 답변에 없는 경우 삭제
    pairs = []
    for i, row in tqdm(train_df.iterrows(), total = len(train_df)):
        for a in range(1, 3):
            if '장점' in row[f'질문_{a}']:
    #             print(i, row[f'질문_{a}'])
                for j in range(1, 6):
                    if '장점' in row[f'답변_{j}']:
                        pair = pd.DataFrame([row[['id', f'질문_{a}', 'category', f'답변_{j}']]])
                        pair.columns = ['id', '질문', 'category', '답변']
                        pairs.append(pair)
            elif '단점' in row[f'질문_{a}']:
    #             print(i, row[f'질문_{a}'])
                for j in range(1, 6):
                    if '단점' in row[f'답변_{j}']:
                        pair = pd.DataFrame([row[['id', f'질문_{a}', 'category', f'답변_{j}']]])
                        pair.columns = ['id', '질문', 'category', '답변']
                        pairs.append(pair)
            else:
                for j in range(1, 6):
                    pair = pd.DataFrame([row[['id', f'질문_{a}', 'category', f'답변_{j}']]])
                    pair.columns = ['id', '질문', 'category', '답변']
                    pairs.append(pair)
    # 모든 쌍을 하나의 DataFrame으로 결합
    train = pd.concat(pairs, ignore_index=True)
    train

    return train

In [4]:
train = cleansing(train_df)
train

  0%|          | 0/644 [00:00<?, ?it/s]

  0%|          | 0/644 [00:00<?, ?it/s]

,id,질문,category,답변
0,TRAIN_000,면진장치가 뭐야?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,TRAIN_000,면진장치가 뭐야?,건축구조,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,TRAIN_000,면진장치가 뭐야?,건축구조,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,TRAIN_000,면진장치가 뭐야?,건축구조,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,TRAIN_000,면진장치가 뭐야?,건축구조,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
...,...,...,...,...
6297,TRAIN_643,"AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?",타 마감하자,"AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기..."
6298,TRAIN_643,"AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?",타 마감하자,"AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기..."
6299,TRAIN_643,"AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?",타 마감하자,"AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기..."
6300,TRAIN_643,"AD, PD에 면한 벽체 결로에 대한 대책은 어떤 것이 있나요?",타 마감하자,"AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기..."


In [5]:
def combine_same_category_questions_equally(data, num_pairs_per_category):
    """
    Combine questions from the same category to create an equal number of new QA pairs for each category.

    Args:
    data (pd.DataFrame): The original dataset containing questions and answers.
    num_pairs_per_category (int): Number of new QA pairs to create per category.

    Returns:
    pd.DataFrame: A DataFrame containing new QA pairs.
    """
    new_qa_pairs = []

    for category in tqdm(data['category'].unique(), total = data['category'].nunique()):
        # 각 id 별로 첫번째 질문-답변 쌍만 사용
        category_data = data[data['category'] == category].drop_duplicates(subset='id')

        # Generate all possible combinations for the category
        all_combinations = list(itertools.combinations(category_data.iterrows(), 2))

        # Randomly select the desired number of combinations, avoiding duplicates
        selected_combinations = random.sample(all_combinations, min(num_pairs_per_category, len(all_combinations)))

        for (idx1, row1), (idx2, row2) in selected_combinations:
            new_question = row1['질문'] + " 또한 " + row2['질문'] + "?"
            new_answer = row1['답변'] + " 또한, " + row2['답변']

            new_qa_pairs.append({'질문': new_question, '답변': new_answer})
            
    return pd.DataFrame(new_qa_pairs)

In [6]:
# Specify the number of pairs per category
num_pairs_per_category = 100

# Create new QA pairs
new_qa_df = combine_same_category_questions_equally(train, num_pairs_per_category)
new_qa_df

  0%|          | 0/7 [00:00<?, ?it/s]

,질문,답변
0,콘크리트 구조는 어떤 방식이 있어? 또한 조적식 구조는 어떤 방식이 있어??,"콘크리트는 철근콘크리트구조, 철골콘크리트구조, 철골철근콘크리트구조, 프리케스트콘크리..."
1,커튼월이 뭐야? 또한 기둥-보 구조 방식이 뭐야??,"커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고, 외벽은 하중을..."
2,콘크리트 구조는 어떤 방식이 있어? 또한 철근콘크리트 구조가 뭐야??,"콘크리트는 철근콘크리트구조, 철골콘크리트구조, 철골철근콘크리트구조, 프리케스트콘크리..."
3,철골구조가 뭐야? 또한 벽돌구조가 뭐야??,철골 구조는 수직의 강철 기둥과 수평의 형강으로 골격을 짜 건물의 하중을 떠받게 하...
4,건축물의 구조는 어떻게 구분해? 또한 제진구조가 뭐야??,"건축물의 구조는 목구조, 조적조, 철골구조, 콘크리트구조로 구별할 수 있습니다. 또..."
...,...,...
695,현관문 주위 결로의 대책은 뭐야? 또한 현관문 주위 결로의 원인은 뭐야??,현관문 주위 결로의 대책은 규격제품을 사용하고 방화문 현관문 내부 종이 허니컴 사이...
696,"부엌 및 욕실의 결로의 원인은 뭐야? 또한 AD, PD에 면한 벽체 결로에 대한 대...",부엌이나 욕실의 경우 외벽이나 비 난방공간에 접한 면이 벽체의 일부인 부엌이나 욕실...
697,발코니에 결로가 발생하는 원인은 뭐야? 또한 벽장 부위 결로의 원인이 뭐야??,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...
698,부엌 및 욕실의 결로의 대책은 뭐야? 또한 외벽 모서리 부위에 결로가 발생하는 원인...,부엌 및 욕실의 결로에 대한 대책은 환기구 설치입니다. 부엌 및 욕실의 결로는 주로...


In [7]:
train = pd.concat([train[['질문', '답변']], new_qa_df[['질문', '답변']]], axis = 0)
train

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,면진장치가 뭐야?,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을..."
2,면진장치가 뭐야?,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...
3,면진장치가 뭐야?,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...
4,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
...,...,...
695,현관문 주위 결로의 대책은 뭐야? 또한 현관문 주위 결로의 원인은 뭐야??,현관문 주위 결로의 대책은 규격제품을 사용하고 방화문 현관문 내부 종이 허니컴 사이...
696,"부엌 및 욕실의 결로의 원인은 뭐야? 또한 AD, PD에 면한 벽체 결로에 대한 대...",부엌이나 욕실의 경우 외벽이나 비 난방공간에 접한 면이 벽체의 일부인 부엌이나 욕실...
697,발코니에 결로가 발생하는 원인은 뭐야? 또한 벽장 부위 결로의 원인이 뭐야??,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...
698,부엌 및 욕실의 결로의 대책은 뭐야? 또한 외벽 모서리 부위에 결로가 발생하는 원인...,부엌 및 욕실의 결로에 대한 대책은 환기구 설치입니다. 부엌 및 욕실의 결로는 주로...


In [8]:
train.to_csv('train_preprocess_data_aug.csv', index=False)